# CAPSTONE PROJECT

In [1]:
import calendar
import pandas as pd
from pandas import Timestamp
from datetime import datetime

In [2]:
CN_HTL = 'hotel'
CN_LT = 'lead_time'
CN_D2 = 'arrival_date_day_of_month'
CN_M2 = 'arrival_date_month'
CN_Y4 = 'arrival_date_year'
CN_ADT = 'adults'
CN_CH = 'children'
CN_CNTY = 'country'
CN_MS = 'market_segment'
CN_AG = 'agent'
CN_CT = 'customer_type'
CN_ADR = 'average_daily_rate'

In [3]:
# Try parse to number
def try_prs_num(s):
    return pd.to_numeric(s, errors='coerce')

In [4]:
# Try parse to number
def try_prs_int(s):
    return pd.to_numeric(s, errors='coerce').astype('Int64')

In [5]:
df = pd.read_excel('portugal_hotel_booking.xlsx')
country_codes_df = pd.read_excel('country_codes_list.xlsx')

In [6]:
hotel = df[CN_HTL]
lead_time = try_prs_int(df[CN_LT])
day = try_prs_int(df[CN_D2])
month = df[CN_M2]
year = try_prs_int(df[CN_Y4])
adults = try_prs_int(df[CN_ADT])
children = try_prs_int(df[CN_CH])
country = df[CN_CNTY]
mkt_seg = df[CN_MS]
agent = try_prs_int(df[CN_AG])
cust_type = df[CN_CT]
avg_dly_rate = try_prs_num(df[CN_ADR]).round(2)

In [7]:
valid_months = set(calendar.month_name[1:])
valids = (lead_time.notna()
              & day.notna()
              & year.notna()
              & adults.notna()
              & children.notna()
              & agent.notna()
              & avg_dly_rate.notna()
              & lead_time.ge(0)
              & day.gt(0)
              & month.isin(valid_months)
              & year.gt(0)
              & adults.ge(0)
              & children.ge(0)
              & ~(adults.eq(0) & ~children.eq(0))
              & agent.ge(0)
              & avg_dly_rate.ge(0)
              & country.isin(country_codes_df['Country'].unique()))

In [8]:
hotel = hotel.loc[valids]
lead_time = lead_time.loc[valids]
day = day.loc[valids]
month = month.loc[valids]
year = year.loc[valids]
adults = adults.loc[valids]
children = children.loc[valids]
country = country.loc[valids]
mkt_seg = mkt_seg.loc[valids]
agent = agent.loc[valids]
cust_type = cust_type.loc[valids]
avg_dly_rate = avg_dly_rate.loc[valids]

In [9]:
month_map = {month_name: month_number for month_number, month_name in enumerate(calendar.month_name) if month_number != 0}

month_num = month.map(month_map)

df_date = pd.DataFrame({
    'year': year,
    'month': month_num,
    'day': day
})
date = pd.to_datetime(df_date, format='%Y-%m-%d', errors='coerce')

In [10]:
# Tukey's fences
def tukey_fences_mod(col_name):
    Q1 = df[col_name].quantile(0.25)
    Q3 = df[col_name].quantile(0.99)
    IQR = Q3 - Q1
    upper_fence = Q3 + 1.5 * IQR
    lower_fence = Q1 - 1.5 * IQR
    return (df[col_name] < lower_fence) | (df[col_name] > upper_fence)

In [11]:
valids &= (pd.notna(date)) & (date < Timestamp.today()) & ~tukey_fences_mod(CN_ADR) & ~tukey_fences_mod(CN_CH)
invalids = ~valids

In [12]:
hotel = hotel.loc[valids]
lead_time = lead_time.loc[valids]
day = day.loc[valids]
month = month.loc[valids]
year = year.loc[valids]
date = date.loc[valids]
adults = adults.loc[valids]
children = children.loc[valids]
country = country.loc[valids]
mkt_seg = mkt_seg.loc[valids]
agent = agent.loc[valids]
cust_type = cust_type.loc[valids]
avg_dly_rate = avg_dly_rate.loc[valids]
weekday = pd.to_datetime(date, format='%Y-%m-%d').apply(lambda x: datetime.strftime(x, '%A'))

In [13]:
pd.DataFrame({
    'Hotel': hotel,
    'Lead Time': lead_time,
    'Arrival Date Day': day,
    'Arrival Date Month': month,
    'Arrival Date Year': year,
    'Arrival Date': date,
    'Weekday': weekday,
    'Adults': adults,
    'Children': children,
    'Country': country,
    'Market Segment': mkt_seg,
    'Agent': agent,
    'Customer Type': cust_type,
    'Average Daily Rate': avg_dly_rate
}).to_csv('data_valid.csv')
df.loc[invalids].to_csv('data_invalid.csv')